# Audio signal presenter

## 0. Setup

### 0.1 Load Packages

In [29]:
import pygame
import time
import os
import random
from collections import Counter

### 0.2 Set random seed to ensure reproducibility

In [18]:
random.seed(42)

### 0.3 Constants

In [19]:
NUM_REPS = 15                
BLOCK_LENGTH = 6            

TAGS = ['dog',
        'environment',
        'human',
        'swine',
        'silence']

### 0.4 Initialize pygame mixer

In [20]:
pygame.mixer.init()
print ("The audio modul is up and running.")

The audio modul is up and running.


### 0.5 Define functions

#### 0.5.1 Sound loader

In [21]:
def sound_loader(tag):
    """
    Load all .wav sounds from the specified folder based on the tag.
    
    Parameters:
    tag (str): The tag for the sounds, which is also the folder name (e.g., dog, pig, human).

    Returns:
    list: A list of preloaded pygame Sound objects.
    """
    folder = f'../data_audio/{tag}'
    sounds_list = []
    
    # Load all .wav files from the specified folder
    for file in os.listdir(folder):
        if file.endswith('.wav'):
            sounds_list.append(os.path.join(folder, file))
    
    # Preload sound files
    preloaded_sounds = [pygame.mixer.Sound(file) for file in sounds_list]
    
    print(f"All {tag} sounds preloaded")
    
    return preloaded_sounds

#### 0.5.2 Pseudorandomizer

In [22]:
def pseudorandomizer(tags = TAGS, num_reps = NUM_REPS):
    '''
    In this context "pseudorandom" simply means random shuffle with no
    two subsequent elements being the same.  
    '''
    total_count = num_reps * len(tags)
    tag_counter = Counter(tags * num_reps)
    
    result = []
    
    def add_tag():
        if len(result) == total_count:
            return True
        
        available_tags = [tag for tag in tags if tag_counter[tag] > 0]
        random.shuffle(available_tags)
        
        for tag in available_tags:
            if len(result) == 0 or result[-1] != tag:
                result.append(tag)
                tag_counter[tag] -= 1
                
                if add_tag():
                    return True
                
                # Backtrack
                result.pop()
                tag_counter[tag] += 1
        
        return False
    
    add_tag()
    return result

#### 0.5.3 Sound player

In [23]:
def play_sound(sound):
    '''
    This function is responsible for playing the sound snippets one
    after the other. 
    '''
    sound.play()
    time.sleep(sound.get_length()) 

## 1. Preload the sounds
This step is crucial to avoid lagging due to buffering

In [24]:
# Preload all the sounds
for tag in TAGS:
    preloaded_sounds = sound_loader(tag)
    globals()[f"{tag}_preloaded_sounds"] = preloaded_sounds
    print(f"{tag}_preloaded_sounds: {preloaded_sounds}")

All dog sounds preloaded
dog_preloaded_sounds: [<pygame.mixer.Sound object at 0x7f48f5b91080>, <pygame.mixer.Sound object at 0x7f48f5b783c0>, <pygame.mixer.Sound object at 0x7f48f5b78270>, <pygame.mixer.Sound object at 0x7f49102b19b0>, <pygame.mixer.Sound object at 0x7f48f5b85170>, <pygame.mixer.Sound object at 0x7f48f5b87cc0>, <pygame.mixer.Sound object at 0x7f48f5b87c00>, <pygame.mixer.Sound object at 0x7f48f5b840f0>, <pygame.mixer.Sound object at 0x7f49104eadf0>, <pygame.mixer.Sound object at 0x7f48f5b56d00>, <pygame.mixer.Sound object at 0x7f48f5b570c0>, <pygame.mixer.Sound object at 0x7f48f5b558c0>, <pygame.mixer.Sound object at 0x7f48f5b56ca0>, <pygame.mixer.Sound object at 0x7f48f5b55950>, <pygame.mixer.Sound object at 0x7f48f5b55740>, <pygame.mixer.Sound object at 0x7f48f5b56880>, <pygame.mixer.Sound object at 0x7f48f5b557a0>, <pygame.mixer.Sound object at 0x7f48f5b567c0>, <pygame.mixer.Sound object at 0x7f48f5b57c30>, <pygame.mixer.Sound object at 0x7f48f5b57b10>, <pygame.mixe

## 2. Create the pseudorandom sequence of blocks
A more proper term could be non-repetitive random or constrained random. But I just stick with pseudorandom. The idea is that the blocks are randomized but two consecutive blocks cannot be the same. 

In [25]:
block_sequence = pseudorandomizer()
print(len(block_sequence), block_sequence)

75 ['swine', 'human', 'swine', 'environment', 'silence', 'human', 'environment', 'silence', 'swine', 'silence', 'swine', 'human', 'dog', 'swine', 'human', 'swine', 'silence', 'dog', 'swine', 'dog', 'swine', 'human', 'environment', 'swine', 'human', 'dog', 'environment', 'silence', 'environment', 'silence', 'dog', 'environment', 'silence', 'swine', 'silence', 'swine', 'silence', 'swine', 'human', 'swine', 'human', 'swine', 'dog', 'swine', 'dog', 'environment', 'human', 'silence', 'dog', 'environment', 'silence', 'environment', 'dog', 'silence', 'human', 'environment', 'silence', 'dog', 'environment', 'human', 'environment', 'human', 'dog', 'silence', 'human', 'environment', 'human', 'dog', 'human', 'silence', 'dog', 'environment', 'dog', 'environment', 'dog']


## 3. Run the experiment

Each block will be 6 seconds long. (Snippets are 0.5 seconds, so 12 of them are needed for one block.) The snippets are selected randomly from the audio pool. There is also a silence block. That's simply generated as 6 seconds of silence.v

In [31]:
# Function to play a sound file and wait for it to finish
print("Experiment starts now.")
for block in block_sequence:
    preloaded_sounds = eval(f"{block}_preloaded_sounds")
    play_this = random.sample(preloaded_sounds, BLOCK_LENGTH*2)
    for snippet in play_this:
        play_sound(snippet)
    print(f"{block} sounds played")
    time.sleep(2)
    print("2 sec pause over")
print("Experiment is over.")

Experiment starts now.
swine sounds played


KeyboardInterrupt: 